In [17]:
import pydantic
print(pydantic.__version__)

2.3.0


In [18]:
from pydantic import BaseModel


class User(BaseModel):
    id: int
    name: str = 'Jane Doe'


In [19]:
user = User(id='123')

In [20]:
print(user.id)

123


In [21]:
print(user.model_fields_set)
user = User(id='123', name='Joe Doe')
print(user.model_fields_set)


{'id'}
{'name', 'id'}


In [22]:
print(user.model_dump())
print(user.model_dump_json())
print(user.model_json_schema())

{'id': 123, 'name': 'Joe Doe'}
{"id":123,"name":"Joe Doe"}
{'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'default': 'Jane Doe', 'title': 'Name', 'type': 'string'}}, 'required': ['id'], 'title': 'User', 'type': 'object'}


### Nested models

In [27]:
from typing import List, Optional
from pydantic import BaseModel


class Food(BaseModel):
    name: str
    price: float
    ingredients: Optional[List[str]] = None


class Restaurant(BaseModel):
    name: str
    location: str
    foods: List[Food]


restaurant_instance = Restaurant(
    name="Tasty Bites",
    location="123, Flavor Street",
    foods=[
        {"name": "Cheese Pizza", "price": 12.50, "ingredients": ["Cheese", "Tomato Sauce", "Dough"]},
        {"name": "Veggie Burger", "price": 8.99}
    ]
)

print(restaurant_instance)
print(restaurant_instance.model_dump())

name='Tasty Bites' location='123, Flavor Street' foods=[Food(name='Cheese Pizza', price=12.5, ingredients=['Cheese', 'Tomato Sauce', 'Dough']), Food(name='Veggie Burger', price=8.99, ingredients=None)]
{'name': 'Tasty Bites', 'location': '123, Flavor Street', 'foods': [{'name': 'Cheese Pizza', 'price': 12.5, 'ingredients': ['Cheese', 'Tomato Sauce', 'Dough']}, {'name': 'Veggie Burger', 'price': 8.99, 'ingredients': None}]}


In [3]:
!pip install pydantic[email]

  Using cached email_validator-2.0.0.post2-py3-none-any.whl (31 kB)
     -------------------------------------- 300.4/300.4 kB 4.6 MB/s eta 0:00:00
  Using cached idna-3.4-py3-none-any.whl (61 kB)



[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from typing import List
from pydantic import BaseModel, EmailStr, PositiveInt, conlist, Field, HttpUrl

class Address(BaseModel):
    street: str
    city: str
    state: str
    zip_code: str

class Employee(BaseModel):
    name: str
    position: str
    email: EmailStr
    
class Owner(BaseModel):
    name: str
    email: EmailStr
    
class Restaurant(BaseModel):
    name: str = Field(..., pattern=r"^[a-zA-Z0-9-' ]+$")
    owner: Owner
    address: Address
    employees: conlist(Employee, min_length=2)
    number_of_seats: PositiveInt
    delivery: bool
    website: HttpUrl

# Creating an instance of the Restaurant class
restaurant_instance = Restaurant(
    name="Tasty Bites",
    owner={
        "name": "John Doe",
        "email": "john.doe@example.com"
    },
    address={
        "street": "123, Flavor Street",
        "city": "Tastytown",
        "state": "TS",
        "zip_code": "12345",
    },
    employees=[
        {
            "name": "Jane Doe",
            "position": "Chef",
            "email": "jane.doe@example.com"
        },
        {
            "name": "Mike Roe",
            "position": "Waiter",
            "email": "mike.roe@example.com"
        }
    ],
    number_of_seats=50,
    delivery=True,
    website="http://tastybites.com"
)

# Printing the instance
print(restaurant_instance)


name='Tasty Bites' owner=Owner(name='John Doe', email='john.doe@example.com') address=Address(street='123, Flavor Street', city='Tastytown', state='TS', zip_code='12345') employees=[Employee(name='Jane Doe', position='Chef', email='jane.doe@example.com'), Employee(name='Mike Roe', position='Waiter', email='mike.roe@example.com')] number_of_seats=50 delivery=True website=Url('http://tastybites.com/')


In [ ]:
from pydantic import BaseModel, EmailStr, field_validator

class Owner(BaseModel):
    name: str
    email: EmailStr
    
    @field_validator('name')
    @classmethod
    def name_must_contain_space(cls, v: str) -> str:
        if ' ' not in v:
            raise ValueError('Owner name must contain a space')
        return v.title()

# Now, let's try creating an instance
try:
    owner_instance = Owner(name="JohnDoe", email="john.doe@example.com")
except ValueError as e:
    print(e)

# This should print: "Owner name must contain a space"
